# Indexing
This jupyter notebook deals with comparing different data structures for the purpose of keeping a database of locations that can be serviced by the drones. For quality of service, it is required that the time taken to search a zone from a pre-exisitng database be minimum.

In [20]:
import random
import time

#generate random words of length n
def generate_name(n):
    name = ""
    for i in range(n):
        name += chr(random.randint(97,122))
    return name

#generate database of N words
def generate_database(N):
    database = []
    for i in range(N):
        database.append(generate_name(random.randint(4,8)))
    return database

In [21]:
generate_database(5)

['nitc', 'evqaqo', 'bptkcpiv', 'krbfec', 'fuymvku']

## Naive approach: $\mathcal{O}(n)$

In [22]:
def search(database, name):
    for element in database:
        if name == element:
            return True
    return False

In [23]:
def computeTime(func):
    N = [10, 100, 1000, 10000, 100000, 1000000]
    for i in N:
        database = generate_database(i)
        date_begin = time.perf_counter()
        isPresent = func(database, "hello")
        date_end = time.perf_counter()
        print("For N = ", i, ", total time:", (date_end*1000000-date_begin*1000000)/1000, "milliseconds, ", isPresent)
    
computeTime(search)

For N =  10 , total time: 0.0036000001430511477 milliseconds,  False
For N =  100 , total time: 0.0085 milliseconds,  False
For N =  1000 , total time: 0.0635 milliseconds,  False
For N =  10000 , total time: 0.6264000000953674 milliseconds,  False
For N =  100000 , total time: 5.644099999904633 milliseconds,  False
For N =  1000000 , total time: 60.25349999976158 milliseconds,  False


## Binary search: $\mathcal{O}(\log n)$
**Disadvantage:** Database needs to be sorted first. Assuming that the database is sorted, then:

In [24]:
def search_binary(database, name):
    start = 0
    end = len(database)-1
    while start <= end:
        middle = (start+end)//2
        if database[middle]==name:
            return True
        elif database[middle]>name:
            end = middle - 1
        else:
            start = middle + 1
    return False

computeTime(search_binary)

For N =  10 , total time: 0.002799999952316284 milliseconds,  False
For N =  100 , total time: 0.0027000002861022948 milliseconds,  False
For N =  1000 , total time: 0.010600000143051147 milliseconds,  False
For N =  10000 , total time: 0.014600000143051147 milliseconds,  False
For N =  100000 , total time: 0.019700000286102293 milliseconds,  False
For N =  1000000 , total time: 0.026799999952316283 milliseconds,  False


## Indexed data structure:
Worst case complexity is $\mathcal{O}(n)$, but for a balanced tree, the average time complexity would be $\mathcal{O}(n)$. The advantage over the binary search is that the database does not need to be sorted.

In [25]:
def build_dict(database):
    dictionary = {chr(i):[] for i in range(97,123)}
    for name in database:
        char = name[0]
        dictionary[char].append(name)
    return dictionary

def search_dict(dictionary, name):
    return search(dictionary[name[0]], name) #or search_binary(dictionary[name[0]], name) if names in dictionary are sorted

In [26]:
def computeTime(func):
    N = [10, 100, 1000, 10000, 100000, 1000000]
    for i in N:
        database = generate_database(i)
        if func==search_dict:
            database = build_dict(database)
        date_begin = time.perf_counter()
        isPresent = func(database, "hello")
        date_end = time.perf_counter()
        print("For N = ", i, ", total time:", (date_end*1000000-date_begin*1000000)/1000, "milliseconds, ", isPresent)
    
computeTime(search_dict)

For N =  10 , total time: 0.004599999904632568 milliseconds,  False
For N =  100 , total time: 0.005900000095367432 milliseconds,  False
For N =  1000 , total time: 0.007100000143051148 milliseconds,  False
For N =  10000 , total time: 0.033699999809265135 milliseconds,  False
For N =  100000 , total time: 0.4119000000953674 milliseconds,  False
For N =  1000000 , total time: 5.177799999952316 milliseconds,  False


If we keep indexing the database into larger sets, for example sets of words beginning from 'aa','ab, ...  It would improve performance, but after a point of time the amount of improvement in the time complexity would not be worth the overhead required to convert the database into a dictionary or the amount of overhead it takes for a computer to build the dictionary in the first place.

## Tree data structure
In this data structure, starting from the root, every branch is indexed by the first letter of a word. For example, all words beggining from the letter 'a' would be in the branch `a`. FOr the next $k$ levels, each branch is indexed by the $(k-1)^{th}$ letter of the word. Finally, each node of the tree are indicated with a booloean, which is `True` if and only if there exists a word starting from the root to that node.

We implement this structure as follows:
The structure is a list of two elements, the first the `True`/`False` value of that node and the second a dictionary where each key is the name of the next element and its value is a similar two element list.

Example: For the words (in french) 'le','ces','cet','cette' we have the structure: `[False, {'c': [False, {'e': [True, {'t': [True, {'t': [False, {'e': [True, {}]}]}], 's': [True, {}]}]}]}]`

The tree is initialized with the value `[False, {}]`


In [27]:
def insert_tree(tree, word):
    if len(word)==1 and word in tree[1]:
        tree[1][word][0] = True
    elif len(word)==1 and not word in tree[1]:
        tree[1][word]=[True, {}]
    else:
        if not word[0] in tree[1]:
            tree[1][word[0]]=[False, {}]	
        insert_tree(tree[1][word[0]], word[1:])

def search_tree(tree, word):
    if word[0] in tree[1]:
        if len(word) == 1 and tree[1][word][0]:
            return True
        elif len(word) != 1:
            return search_tree(tree[1][word[0]], word[1:])
    return False

In [28]:
def computeTime(func):
    N = [10, 100, 1000, 10000, 100000, 1000000]
    for i in N:
        database = generate_database(i)
        if func == search_dict:
            database = build_dict(database)
        if func == search_tree:
            tree = [False, {}]
            for word in database:
                insert_tree(tree, word)
            database = tree
        date_begin = time.perf_counter()
        isPresent = func(database, "hello")
        date_end = time.perf_counter()
        print("For N = ", i, ", total time:", (date_end*1000000-date_begin*1000000)/1000, "milliseconds, ", isPresent)
    
computeTime(search_tree)

For N =  10 , total time: 0.0037000000476837156 milliseconds,  False
For N =  100 , total time: 0.0033999998569488525 milliseconds,  False
For N =  1000 , total time: 0.007099999904632569 milliseconds,  False
For N =  10000 , total time: 0.0065 milliseconds,  False
For N =  100000 , total time: 0.009700000047683717 milliseconds,  False
For N =  1000000 , total time: 0.03339999985694885 milliseconds,  False


## Indexing on multiple letters
Up till now, we have been searching for words in a database. However, if we were to change the question to "Which words in the database start with the letters xyz?" then it would be difficult to answer this question based on the above approaches.

Below, we define a new data structure which can help in answering this question. This datastructure is called a [trie](https://en.wikipedia.org/wiki/Trie). This implementation is taken from [towardsdatascience](https://towardsdatascience.com/implementing-a-trie-data-structure-in-python-in-less-than-100-lies-of-code-a877ea23c1a1). The only change involves keeping a set of words for each node which tells us the words inand below that node.

In [29]:
# simple method to find which words contains the letter 'x'
def dictContainsX(database):
    dictionary = {chr(i):set() for i in range(97,123)} # use set to avoid repitition
    for name in database:
        for letter in name:
            if not name in dictionary[letter]:
                dictionary[letter].add(name)
    return dictionary

In [30]:
class TrieNode():
    def __init__(self, char: str):
        self.char = char
        self.children = []   # child nodes
        self.word_end = False    # if the word ends at this node
        self.words = set() # words in and below this node
        
def add(root, word: str):
    node = root
    for char in word:
        found = False
        for child in node.children:
            if child.char == char:
                child.words.add(word)
                node = child
                found = True
                break
                
        if not found:
            new_node = TrieNode(char)
            new_node.words.add(word)
            node.children.append(new_node)
            node = new_node
            
    node.word_end = True

def find_prefix(root, prefix: str):
    node = root
    
    if not root.children:
        return False, set()
    for char in prefix:
        char_not_found = True
        for child in node.children:
            if child.char == char:
                char_not_found = False
                node = child
                break
        if char_not_found:
            return False, set()
    return True, node.words

In [31]:
root = TrieNode('*')
database = generate_database(100)
for word in database:
    add(root, word)

In [32]:
total = 0
for c in range(97,123):
    _, set_c = find_prefix(root, chr(c))
    total += len(set_c)
print('total words: ', total)

total words:  100
